In [110]:
# import
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from itertools import repeat

import numpy as np
import pandas as pd
from Utils import custom_data_loader, preprocess_data
from Utils.SummaryWriter import LogSummary
from Models.simpleFFBNN import SimpleFFBNN
from Models.denseRegression import DenseRegressor



In [2]:
def get_device():
    """Function to get the device to be used for training the model
    """
    cuda = torch.cuda.is_available()
    print("CUDA Available: ", cuda)

    if cuda:
        gpu = GPUtil.getFirstAvailable()
        print("GPU Available: ", gpu)
        torch.cuda.set_device(gpu)
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    print("Device: ", device)
    return device

device = get_device()


CUDA Available:  False
Device:  cpu


In [3]:
# load data
dataloader_train, dataloader_test, dataloader_val = preprocess_data(pd.read_csv('/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/Data/quality_of_food.csv'), batch_size = 128)


/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/Utils/config.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.X['savings'] = np.where(self.X['savings'] == 'low', 0, np.where(self.X['savings'] == 'medium', 1, 2))


In [7]:
model = SimpleFFBNN(4, 1)

In [18]:
checkpoint = torch.load('/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/trainedModels/simple_model.pth', map_location=torch.device('cpu'))
#print(checkpoint)
#model.load_state_dict(checkpoint['model'])
model.load_state_dict(checkpoint)
    

<All keys matched successfully>

In [19]:
class Saveoutput():
    def __init__(self, instances, batch_size, rounds):
        self.T = instances
        self.batch_size = batch_size
        self.outputs = []
        self.rounds = rounds
        self.counter = 0


    def __call__(self, module, module_in, module_out):
        if self.counter < 3:
            sample_data = np.random.randint(self.batch_size)
            outs = module_out.view(self.batch_size, -1)
            #outs = module_out.view(self.T, self.batch_size, -1)[:, 0, :]
            layer_size = outs.shape[1]

            
            write_summary.per_round_layer_output(layer_size, outs, self.rounds)

            self.counter += 1


    def clear(self):
        self.outputs = []
        


        

In [15]:
write_summary = LogSummary(name = 'Simple Model')

In [132]:
class runActiveLearning():
    def __init__(self, model_name, model, dataloader_train, dataloader_test, dataloader_val, epochs, rounds, learning_rate, batch_size, instances, highest_unc, seed_sample, retrain, resume_round):
        self.model_name = model_name
        self.model = model
        self.dataloader_train = dataloader_train
        self.dataloader_test = dataloader_test
        self.dataloader_val = dataloader_val
        self.epochs = epochs
        self.rounds = rounds
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.instances = instances
        self.highest_unc = highest_unc
        self.seed_sample = seed_sample


        # a set of lists to store the selected indices with highest uncertainty
        self.highest_unc = set([])
        # unexplored data
        self.unexplored_data = set(range(len(dataloader_train.dataset)))

        if resume_round and not retrain:
            self.InitModel(load_weigths=True, res_round=resume_round)
        elif resume_round and retrain:
            self.InitModel(load_weigths=False, res_round=resume_round)
        else:
            self.InitModel()


        training_params = sum(p.numel() for p in self.model.parameters())
        print('Running Model:{}'.format(model, training_params, self.epochs, self.batch_size))


    def InitModel(self,load_weigths = False, res_round = None):
        if self.model_name == 'simple':
            self.model = SimpleFFBNN(4, 1, )
        else:
            self.model = DenseRegressor(4, 1)
            
    def loadSeedModel(self):
        # load the trained model
        model = SimpleFFBNN(4, 1)
        checkpoint = torch.load('Activelearning/SeedModels/simple_model_best.pth', map_location=torch.device('cpu'))
        model.load_state_dict(checkpoint)
        self.model = model
        print(f'Model loaded: {model}')

    def loadPreTrainedModel(self, load_weigths, res_round):
        self.getTrainedModel(res_round)
        # load weights
        if DEVICE.type == 'cpu':
            state = torch.load(self.train_weight_path, map_location=torch.device('cpu'))
        else:
            state = torch.load(self.train_weight_path)

        self.highest_unc = state['highest_unc']
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

        if load_weigths:
            self.model.load_state_dict(state['weights'])
            self.optimizer.load_state_dict(state['optimizer'])

    def getEntropy(self, y):
        ensemble_probabilities = y.mean(0)
       # print(f'Ensemble probabilities func: {ensemble_probabilities.shape}')
        entropy = Categorical(probs = ensemble_probabilities).entropy()
        # get the entropy of the ensemble probabilities
        #entropy = -torch.sum(ensemble_probabilities * torch.log(ensemble_probabilities), dim=1)

        #probabilities = Categorical(probs = ensemble_probabilities)

       # print(f'Probabilities func: {probabilities.probs[0:5]}')

        #entropy = probabilities.entropy()

        #entropy  = -torch.sum(ensemble_probabilities * torch.log(ensemble_probabilities), dim=1)

        #print(f'Entropy func: {entropy[0:5]}')
        return entropy


    def trainSeedModelClosedForm(self, rounds):
        # loss function
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)


        if rounds == 1:
            self.highest_unc = set(range(self.seed_sample))
            self.unexplored_data = self.unexplored.difference(self.highest_unc)



        uncertainty = []
        hook_handles = []
        save_output = Saveoutput(instances= self.instances, batch_size=self.batch_size, rounds = self.rounds)
        self.model.eval()
        for layer in self.model.kl_layers:
            hook_handles.append(layer.register_forward_hook(save_output))

        with torch.no_grad():
            for batch_index, (X, y) in enumerate(self.dataloader_train):
                batch_size = X.shape[0]
                #print(f'X before repeat: {X}')
               # print(f'y before repeat: {y}')
                save_output.batch_size = batch_size
                X = X.repeat(self.instances, 1)
                #print(f'X after repeat: {X}, {X.shape}')
                

                y = y.squeeze()
                # squeeze the y to remove the extra dimension
                
                y = y.repeat(self.instances)
        
                X, y = X.to(device), y.to(device)
                y_pred = model(X)
                print(f'y_pred: {y_pred.shape}, {y_pred[0:5]}, {y_pred[128:133]}')
                
                

                ensemble_outputs = y.reshape(self.instances, batch_size, 1) # reshape to instances x batch_size x 1 (output size)
                
                
                uncertainty.append(self.getEntropy(ensemble_outputs))
                   
        
            save_output.clear()
            save_output.counter = 0
            for handle in hook_handles:
                handle.remove()
            
            uncertainty = torch.cat(uncertainty)
            
            new_indices = torch.argsort(uncertainty, descending=True).tolist()

            # remove already selected data
            new_indices = [i for i in new_indices if i not in self.highest_unc]  

            # select the highest uncertainty data
            self.highest_unc.union(set(new_indices[:self.seed_sample]))
            print(f'higheset_unc: {self.highest_unc}')
            self.unexplored_data = self.unexplored_data.difference(self.highest_unc)

    
    def objective(self, output, target, kl, beta):
        loss_fun = nn.MSELoss()
        discrimination_error = loss_fun(output.view(-1), target.view(-1))
        variational_bound = discrimination_error + beta * kl
        return variational_bound, discrimination_error, kl


    def getTrainedModel(self, res_round):
        retrain = self.retrain
        self.train_weight_path = f'Activelearning/SeedModels/simple_model_best.pth'
        if res_round:
            self.train_weight_path = f'Activelearning/SeedModels/simple_model_best.pth'
        else:
            self.train_weight_path = f'Activelearning/SeedModels/simple_model_best.pth'

        return (self.model, self.train_weight_path)



    def trainModel(self, rounds, epochs):
        t_total, v_total = 0, 0
        t_r2_scores, v_r2_scores = [], []
        self.model.train()
        t_loss, v_loss = [], []
        t_likelihood, v_likelihood = [], []
        t_kl, v_kl = [], []
        self.model.train()
        m = len(self.dataloader_train)

        for batch_index, (inputs, targets) in enumerate(self.dataloader_train):
            X = inputs.repeat(1, 1)  # (number of mcmc samples)
            Y = targets.repeat(1)
            X, Y = X.to(device), Y.to(device)
            outputs = self.model(X)
            loss, log_likelihood, kl = self.objective(outputs, Y, self.model.kl_divergence, m)
        
        t_total += target.size(0)
        t_r2_scores.append(r2_score(outputs.detach().cpu()[:,0], targets.numpy()))
        t_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        for layer in self.model.kl_layers:
            layer.clip_variances()

        
        # validate
        m_val = len(self.dataloader_val)
        self.model.eval()
        for batch_index, (inputs, targets) in enumerate(self.dataloader_val):
            X = self.model(inputs)
            loss_val, log_likelihood_val, kl_val = self.objective(outputs, Y, self.model.kl_divergence, m_val)
        
            v_total += target.size(0)
            v_loss.append(loss_val.item())
            v_likelihood.append(log_likelihood_val.item())
            v_kl.append(kl_val.item())
           # v_r2_scores.append(r2_score(outputs.detach().cpu()[:,0], targets.numpy()))
            


            

   # write_summary = LogSummary(name='Simple Model')





# use the class to run the active learning
active_learning = runActiveLearning(model_name='simple', model=model, dataloader_train=dataloader_train, dataloader_test=dataloader_test, dataloader_val=dataloader_val, epochs=100, rounds=2, learning_rate=0.001, batch_size=128, instances=3, highest_unc=6, seed_sample=4, retrain=False, resume_round=False)

Running Model:SimpleFFBNN(
  (fc1): KlLayers (4 -> 10)
  (fc2): KlLayers (10 -> 20)
  (fc3): KlLayers (20 -> 1)
)


In [135]:
active_learning.trainSeedModelClosedForm(rounds = 2)

y_pred: torch.Size([384, 1]), tensor([[ 0.3558],
        [ 0.2122],
        [-0.8390],
        [ 0.2063],
        [ 0.1837]]), tensor([[ 0.3558],
        [ 0.2122],
        [-0.8390],
        [ 0.2063],
        [ 0.1837]])
y_pred: torch.Size([384, 1]), tensor([[-0.1449],
        [-0.9519],
        [-1.5050],
        [-0.1934],
        [ 0.0758]]), tensor([[-0.1449],
        [-0.9519],
        [-1.5050],
        [-0.1934],
        [ 0.0758]])
y_pred: torch.Size([384, 1]), tensor([[-0.3341],
        [-1.1289],
        [-0.8976],
        [ 0.7849],
        [ 0.3607]]), tensor([[-0.3341],
        [-1.1289],
        [-0.8976],
        [ 0.7849],
        [ 0.3607]])
y_pred: torch.Size([384, 1]), tensor([[ 0.2712],
        [ 0.7198],
        [ 0.6477],
        [-0.4300],
        [ 0.0229]]), tensor([[ 0.2712],
        [ 0.7198],
        [ 0.6477],
        [-0.4300],
        [ 0.0229]])
y_pred: torch.Size([384, 1]), tensor([[-0.4118],
        [-0.1848],
        [ 0.4311],
        [-1.1234],
  

In [8]:
def trainSeedModelClosedForm(self):
    hook_handles = []
    save_output = saveOutput(self.instances, self.batch_size, 0)
    self.model.eval()
    for layer in self.model.layers:
        hook_handles.append(layer.register_forward_hook(save_output))
    
    with torch.no_grad():
        for batch_index, (X, y) in enumerate(self.data_train):
            batch_size = X.shape[0]
            save_output.batch_size = batch_size
            X = X.repeat(self.instances, 1)
            y = y.repeat(self.instances)
            X, y = X.to(self.device), y.to(self.device)
            ensemble_outputs = self.model(X)

        save_output.clear()
        save_output.counter = 0
        for handle in hook_handles:
            handle.remove()
